In [4]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
from numpy import exp
from numpy import sin
from numpy import deg2rad
from numpy.linalg import eig

In [207]:
# SNR = 20
# snapshots = 200
# sensorsNum = 8
def ESPRIT(SNR = 20, snapshots = 200, sensorsNum = 8):
#----------Consider a ULA, where the array sapcing is a half wavelength of the signal.--------#
    c = 3e8
    f = 2.4e9                       # frequency is 2.4GHz
    wavelength = c / f              # lambda
    spacing = wavelength / 2        # ULA's spacing

#---------Sample: sample frequency is fs = 3f-------------------------------------------#
    fs = 3 * f                      
    Ts = 1 / fs                             # Sample period
    Ns = Ts * np.arange(snapshots)          # Sample spacing

#----------Consider noises are generated from a zero mean Gaussian distribution.--------------#
    sigma_N = 0.1
    noiseCovMat = np.diag(sigma_N * np.ones(sensorsNum))    

    noiseAmp = np.random.multivariate_normal(np.zeros(sensorsNum), noiseCovMat, snapshots)
    noisePhase = np.mat([exp(-1j*2*pi*f*Ns + np.random.rand())])
    noiseMat = np.multiply(noiseAmp, noisePhase.T)                    # Each row is A sample 

#----------Consider four uncorrlated sources at -25 degree, 0 degree and 25 degree,-------#
#----------Each source is generated from a zerom mean Gaussian distribution.------------------#    
    theta_S = np.array([-25, 0, 25])
    sourcesNum = len(theta_S)
    sigma_S = sigma_N * 10**(SNR/20)
    signalCovMat = np.diag(sigma_S * np.ones(sourcesNum))
    signalAmp = np.random.multivariate_normal(np.zeros(sourcesNum), signalCovMat, snapshots)
    signalPhase = np.mat([exp(-1j*2*pi*f*Ns + np.random.rand())])
    signalMat = np.multiply(signalAmp, signalPhase.T)                # Each row is A sample 

    spacingK = spacing * np.arange(sensorsNum)
    manifoldMat = np.zeros((sensorsNum, sourcesNum), dtype=complex)
    for col in range(manifoldMat.shape[1]):
            manifoldMat[:, col] = np.mat([exp(-1j*2*pi*f*((spacingK*np.sin(np.deg2rad(theta_S[col])))/c))], dtype=complex)

    spacingSubspace = 1 * spacing
    phaseDelayMat = np.diag(exp(1j*(2*pi*spacingSubspace*sin(deg2rad(theta_S)))/wavelength))

    sensorsOut1 = np.dot(manifoldMat, signalMat.T) + noiseMat.T    # X1 = AS + N, each column is A sample
    sensorsOut2 = np.dot(np.dot(manifoldMat, phaseDelayMat), signalMat.T) + noiseMat.T    # X2 = A$\Phi$S + N, each column is A sample
    sensorOut = np.vstack((sensorsOut1, sensorsOut2))

    covMat_hat = np.dot(sensorOut, sensorOut.H) / snapshots

    eigenValues, eigenVectors = eig(covMat_hat)
    eigenValuesIdx = np.argsort(-eigenValues)
    signalSubspace = eigenVectors[:, eigenValuesIdx[range(sourcesNum)]]
    signalSubspace1 = signalSubspace[range(sensorsNum), :]
    signalSubspace2 = signalSubspace[range(sensorsNum, 2*sensorsNum), :]
    constructMat = np.hstack((signalSubspace1, signalSubspace2))
    eigvals, eigVecMat = np.linalg.eig(np.dot(constructMat.H, constructMat))
    subMat12 = eigVecMat[0: sourcesNum, range(sourcesNum, 2*sourcesNum)]
    subMat22 = eigVecMat[sourcesNum: 2*sourcesNum, range(sourcesNum, 2*sourcesNum)]
    Psi = -np.dot(subMat12, subMat22.I)
    phase_hat = np.linalg.eigvals(Psi)
    theta_hat = np.rad2deg(np.arcsin((np.angle(phase_hat)*wavelength)/(2*pi*spacingSubspace)))
    theta_hat.sort()
    return theta_hat
#     print(theta_hat)

[-1.97185824e+01 -3.66995725e-15  1.95957241e+01]
